In [1]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)
path = 'gdrive/MyDrive/MadeProject'

Mounted at /content/gdrive/


In [ ]:
# !git clone https://github.com/NVIDIA/DeepLearningExamples
!git clone https://github.com/GimmeDanger/made-rec-sys-2021.git
ncf_path = '/content/made-rec-sys-2021/ncf'

In [ ]:
# intall requirements: apex + ncf
!cd '{ncf_path}' && python3 -m pip install -r requirements.txt

# https://stackoverflow.com/questions/66610378/unencryptedcookiesessionfactoryconfig-error-when-importing-apex
!git clone https://github.com/NVIDIA/apex
!cd apex && python3 -m pip install -v --disable-pip-version-check --no-cache-dir \
--global-option="--cpp_ext" --global-option="--cuda_ext" ./

In [7]:
import pandas as pd

#interaction_path = 'interaction_by_orders.parquet'
interaction_path = 'interaction_by_orders_small.parquet'
#interaction_path = 'ncf_orders.parquet'

!mkdir -p /data
!mkdir -p /data/cache/interactions

df = pd.read_parquet(f'{path}/{interaction_path}')
df.to_csv('/data/interactions.csv')
# weight is dummy
df.head()

,user_id,chain_id,weight
index,,,
707,3209,33446,3
2463,10803,15275,1
2464,10803,28720,2
3859,16781,15734,2
3860,16781,28309,1


In [8]:
!cd '{ncf_path}' && python3 convert.py \
    --path /data/interactions.csv \
    --output /data/cache/interactions \
    --valid_negative=100

Loading raw data from /data/interactions.csv
Mapping original user and item IDs to new sequential IDs
3411 3841
Generating validation negatives...
100% 384100/384100 [00:00<00:00, 1970609.23it/s]


In [9]:
!cd '{ncf_path}' && python3 -m torch.distributed.launch --nproc_per_node=1 \
    --use_env ncf.py --data /data/cache/interactions \
    --checkpoint_dir /data/checkpoints/

/usr/local/lib/python3.7/dist-packages/torch/distributed/launch.py:186: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use_env is set by default in torchrun.
If your script expects `--local_rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  FutureWarning,
DLL 2021-11-27 14:21:13.385279 - PARAMETER data : /data/cache/interactions  feature_spec_file : feature_spec.yaml  epochs : 30  batch_size : 1048576  valid_batch_size : 1048576  factors : 64  layers : [256, 256, 128, 64]  negative_samples : 4  learning_rate : 0.0045  topk : 10  seed : None  threshold : 1.0  beta1 : 0.25  beta2 : 0.5  eps : 1e-08  dropout : 0.5  checkpoint_dir : /data/checkpoints/  load_checkpoint_path : None  mode : train  grads_accumulated : 1  amp : False  log_path : log.json  world_size : 1  distributed :

In [10]:
!cd '{ncf_path}' && python3 -m torch.distributed.launch --nproc_per_node=1 \
    --use_env ncf.py -k 30 --data /data/cache/interactions \
    --mode test --load_checkpoint_path /data/checkpoints/model.pth

/usr/local/lib/python3.7/dist-packages/torch/distributed/launch.py:186: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use_env is set by default in torchrun.
If your script expects `--local_rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  FutureWarning,
DLL 2021-11-27 14:21:24.941838 - PARAMETER data : /data/cache/interactions  feature_spec_file : feature_spec.yaml  epochs : 30  batch_size : 1048576  valid_batch_size : 1048576  factors : 64  layers : [256, 256, 128, 64]  negative_samples : 4  learning_rate : 0.0045  topk : 30  seed : None  threshold : 1.0  beta1 : 0.25  beta2 : 0.5  eps : 1e-08  dropout : 0.5  checkpoint_dir :   load_checkpoint_path : /data/checkpoints/model.pth  mode : test  grads_accumulated : 1  amp : False  log_path : log.json  world_size : 1  distribut